# Global effect

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt




import pandas as pd

with open('/home/23m1526/ua.base', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()


data = []
for line in lines:
    parts = line.strip().split('\t')
    user_id = int(parts[0])
    item_id = int(parts[1])
    rating = int(parts[2])

    data.append((user_id, item_id, rating))


df = pd.DataFrame(data, columns=['userId', 'movieId', 'rating'])

# Print the first few rows of the DataFrame
df


df['movieId'].max()
df['movieId'].min()
temp = 1
userItem  = np.zeros((df['userId'].max()+1,df['movieId'].max()+1))
print(userItem )
print(userItem.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(944, 1683)


In [4]:
for i in range (len(df['userId'])):
  userItem[df.userId[i]][df.movieId[i]] = df.rating[i]


In [5]:
R = userItem[1:]

R = R[:, 1:]

ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]

In [6]:

import math
def RMSE(actual, predicted):
    err = 0
    
    count = np.count_nonzero(actual)
    for i in range(actual.shape[0]):
        for j in range(actual.shape[1]):
            if actual[i][j] > 0:
                
                err += (actual[i][j] - predicted[i][j])**2
    
    if count == 0: "error is not calculated therefore count = 0 "
    else : 
        mse = err/count
    rmse = math.sqrt(mse)
    
    return  rmse
    

In [7]:
def evaluate_global_average(R, epsilon1):
    # Calculate global average
    global_avg = (np.sum(R) + np.random.laplace(5-1/epsilon1)) / 1e5
    

    return global_avg


In [8]:
import numpy as np



def evaluate_item_averages(R,A, beta_i, epsilon1, epsilon2 ):
    
    r_min, r_max = 1,4
    # Calculate global average
    global_avg = (np.sum(R) + np.random.laplace(5-1/epsilon1)) / 1e5

    # Initialize item averages
    item_averages = np.zeros(R.shape[1])
    
    Rk = np.sum(A, axis=0) # no of rating given to a item    
 
    for j in range(R.shape[1]):
        # Select ratings for item j
        Rj = R[:, j]

        # Calculate item average
        item_avg = (np.sum(Rj) + beta_i * global_avg + np.random.laplace(scale= 4 /epsilon2)) / (Rk[j] + beta_i)

        # Clamp item average to specified range
        item_avg = np.clip(item_avg, 1, 5 )

        item_averages[j] = item_avg

    return item_averages


In [13]:


A = R>0.5; A[A == True] = 1; A[A == False]   = 0; A = A.astype(np.float64, copy=False)
ItemAvg = evaluate_item_averages(R,A,  0.5,0.1, 0.5)
print(ItemAvg)
Rj = np.sum(A, axis=0) # no of rating given to a item

[3.85261648 3.27821521 3.15915038 ... 1.         1.         5.        ]


In [20]:

rmse_IAvg = []
ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for ep in ep:
    Item_avg = evaluate_item_averages(R,A,  10,0.1, 0.5)  # Compute global average for current epsilon
      # Create a matrix filled with the global average
    A = np.tile(np.array(Item_avg), (R.shape[0], 1))
        
    rmse_temp = RMSE(R, A)  # Now G is a matrix of the same shape as R
    rmse_IAvg.append(rmse_temp)


In [21]:
print(rmse_IAvg)

[2.719033996797629, 2.7473327415860127, 2.7190649077276867, 2.7473985464434265, 2.7188335280547253, 2.747424745615418, 2.719231388266216, 2.7473554583894733, 2.719155152394188, 2.7470024100182564]


In [99]:
import numpy as np

def evaluate_user_averages(R,A, item_avgs, beta_u, epsilon_1, epsilon_2):
    Ri = np.sum(A, axis=1)# no of rating given by a user    
    
    R0 = R - item_avgs
    R2 = np.zeros_like(R0)
     
# Assign values from R0 where R1 is 1
    R2[A == 1] = R0[A == 1]
    R0 = R2

    n_users, m_items = R.shape
    # Calculate the new global average based on adjusted ratings
    global_avg_0 = np.mean(R0) + np.random.laplace(scale=2 * np.max(np.abs(R)) / epsilon_1)/(10**5 )
    
    
    
    user_avgs = np.zeros(n_users)

    for v in range(n_users):
        # Select ratings for user v
        ratings_v = R0[v, :]
        # Calculate user average
   
        # Add fictitious ratings with the new global average for each user
        user_avg = (np.sum(ratings_v) + beta_u * global_avg_0 + np.random.laplace(scale=4 / epsilon_2))/(Ri[v] + beta_u)
        # Clamp user average to [-2, 2]
        user_avgs[v] = np.clip(user_avg, -2, 2)

    return user_avgs

In [100]:
UserAvg = evaluate_user_averages(R,A, ItemAvg, 0.1, 0.2, 0.3)
print(UserAvg)

[ 1.61292110e-01 -7.41818907e-02 -6.83709148e-01  9.80186026e-01
 -4.02952558e-01 -3.16785534e-01  4.53931817e-01  3.76140264e-01
 -7.01203059e-02  4.24251951e-01 -4.60302988e-02  3.49862247e-02
 -2.40330016e-01  4.61477515e-01 -5.05503420e-01  5.88388583e-01
 -4.57259546e-01  1.46278362e-01 -2.00000000e+00  9.39991840e-01
 -2.73781869e-01 -1.69858006e-02 -3.88363076e-02  6.09322074e-01
  5.32054884e-02  1.12794117e-03  4.37744071e-01  2.92288099e-01
  2.24045572e-01  2.54225539e-01 -8.54538479e-02 -8.35116798e-01
  1.01855442e+00  2.21962537e-01 -1.20269345e-01  2.00000000e+00
  2.15835660e-01  2.33510899e-01  1.91593442e+00 -8.27568307e-01
 -6.08093753e-02  5.26170596e-02  1.82947583e-01  1.31287081e-01
  2.05573467e-01  1.33932878e+00  2.00000000e+00 -2.05486682e-01
 -6.18089681e-01  1.48379383e-01 -6.93863324e-01  1.60746794e+00
  1.18084798e-01  7.75345459e-01  7.24202602e-01  1.90480507e-01
  1.71271623e-01  8.66932581e-02  2.78309457e-01  6.30484240e-01
 -1.85603527e+00 -2.22993

In [101]:
def rating_globle_effect(U,I):
  ratings = U[:, np.newaxis] + I
  return ratings
GAvg = rating_globle_effect(UserAvg,ItemAvg)
print(GAvg)


[[4.04868209 3.37065262 3.04165662 ... 1.39703928 1.16129211 5.16129211]
 [3.81320809 3.13517862 2.80618261 ... 1.16156528 0.92581811 4.92581811]
 [3.20368084 2.52565136 2.19665536 ... 0.55203802 0.31629085 4.31629085]
 ...
 [5.88738998 5.20936051 4.88036451 ... 3.23574717 3.         7.        ]
 [4.70130944 4.02327997 3.69428396 ... 2.04966662 1.81391946 5.81391946]
 [3.75926826 3.08123879 2.75224279 ... 1.10762545 0.87187828 4.87187828]]


In [103]:
GEff_Rmse = []

ep = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
for ep in (ep):
    I = evaluate_item_averages(R,A ,20,0.2, ep)
    U = evaluate_user_averages(R,A, I,10,0.2,ep)
    G = rating_globle_effect( U ,I)      
    temp = RMSE(R,G)
    GEff_Rmse.append(temp)

In [104]:
GEff_Rmse

[0.9266320010658061,
 0.9254640160901215,
 0.9253576700915123,
 0.9253303607628602,
 0.9252877848048937,
 0.9252188989665445,
 0.9252251174760471,
 0.9252095197037631,
 0.9252010061793053,
 0.925201169079698]

In [107]:
import csv



csv_file_path = 'output.csv'

In [108]:
with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    writer.writerow(['GEff_Rmse'] + GEff_Rmse)